In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
import re

BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

def get_prodi_info(prodi_id):
    """
    Mengambil nama prodi dan jumlah halaman maksimum dalam satu kali request.
    """
    url = f"{BASE_URL}/{prodi_id}/1"
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")

        # 1. Dapatkan nama prodi
        prodi_name_element = soup.select_one('div#begin h2')
        if not prodi_name_element:
            return None, 0 # Elemen nama prodi tidak ditemukan
        
        prodi_full = prodi_name_element.text.strip()
        prodi_name = prodi_full.replace("Journal Jurusan ", "")

        # 2. Dapatkan jumlah halaman
        last_page_link = soup.select_one('ol.pagination a:contains("»")')
        if last_page_link and "href" in last_page_link.attrs:
            href = last_page_link["href"]
            max_page = int(href.split("/")[-1])
        else:
            max_page = 1 # Fallback jika pagination tidak ada
            
        return prodi_name, max_page
        
    except requests.exceptions.RequestException as e:
        print(f"\nError saat mengakses informasi prodi {prodi_id}: {e}")
        return None, 0

def print_progress(prodi, current_page, total_pages):
    """Menampilkan progress bar di console."""
    percent = (current_page / total_pages) * 100
    bar_length = 30
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\rScraping {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()


def scrape_prodi(prodi_id):
    start_time = time.time()
    
    prodi_name, max_page = get_prodi_info(prodi_id)
    if not prodi_name:
        print(f"Gagal mendapatkan informasi untuk prodi ID: {prodi_id}. Proses dihentikan.")
        return None
        
    print(f"Prodi terdeteksi: {prodi_name} (ID: {prodi_id})")
    print(f"Ditemukan {max_page} halaman untuk di-scrape.")
    
    data = {
        "id": [], "penulis": [], "judul": [], "abstrak_id": [], "abstrak_en": [],
        "pembimbing_pertama": [], "pembimbing_kedua": [], "prodi": []
    }

    for page in range(1, max_page + 1):
        url = f"{BASE_URL}/{prodi_id}/{page}"
        try:
            r = requests.get(url)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            for jurnal in jurnals:
                link_detail = jurnal.select_one('a.gray.button')['href']
                id_match = re.search(r"/detail/(\d+)", link_detail)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link_detail)
                soup_detail = BeautifulSoup(response.content, "html.parser")
                isi = soup_detail.select_one('div#content_journal')
                
                if not isi: continue

                # --- FUNGSI HELPER PALING ROBUST UNTUK HTML ANDA ---
                def get_metadata_value_robust(soup_content, label):
                    element = soup_content.select_one(f'span:-soup-contains("{label}")')
                    if element:
                        # Ambil teks dari elemen induknya (div) lalu bersihkan
                        parent_text = element.parent.get_text(strip=True)
                        # Pisahkan berdasarkan titik dua pertama, lalu ambil bagian kedua
                        parts = parent_text.split(':', 1)
                        if len(parts) > 1:
                            return parts[1].strip()
                    return "N/A" # Kembalikan N/A jika tidak ditemukan

                judul = isi.select_one('a.title').text.strip()
                penulis = get_metadata_value_robust(isi, "Penulis")
                pembimbing_pertama = get_metadata_value_robust(isi, "Dosen Pembimbing I")
                pembimbing_kedua = get_metadata_value_robust(isi, "Dosen Pembimbing II")
                
                paragraf = isi.select('p[align="justify"]')
                abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                # --- AKHIR DARI BLOK LOGIKA ---

                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["abstrak_id"].append(abstrak_id)
                data["abstrak_en"].append(abstrak_en)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["prodi"].append(prodi_name)

            print_progress(prodi_name, page, max_page)
        
        except requests.exceptions.RequestException as e:
            print(f"\nError pada halaman {page} untuk prodi {prodi_name}: {e}")
            continue

    if not data["id"]:
        print("\nTidak ada data yang berhasil di-scrape.")
        return None

    df = pd.DataFrame(data)
    output_filename = f"pta_{prodi_name.lower().replace(' ', '_')}_{prodi_id}.csv"
    df.to_csv(output_filename, index=False, encoding="utf-8-sig")
    
    end_time = time.time()
    elapsed = int(end_time - start_time)
    menit, detik = divmod(elapsed % 3600, 60)
    jam = elapsed // 3600

    print(f"\n\n✅ Selesai! Data prodi {prodi_name} berhasil dikumpulkan.")
    print(f"📄 Data disimpan dalam file: {output_filename}")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

# --- Cara Menjalankan ---
if __name__ == "__main__":
    # Anda cukup memasukkan ID prodi yang diinginkan
    ID_PRODI_TARGET = 7
    
    df_hasil = scrape_prodi(ID_PRODI_TARGET)

    if df_hasil is not None:
        print("\nContoh 5 data pertama yang diambil:")
        print(df_hasil.head())

C:\laragon\bin\python\python-3.10\lib\site-packages\soupsieve\css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


Prodi terdeteksi: Manajemen (ID: 7)
Ditemukan 207 halaman untuk di-scrape.



Error pada halaman 1 untuk prodi Manajemen: HTTPSConnectionPool(host='pta.trunojoyo.ac.id', port=443): Read timed out. (read timeout=None)


Scraping Manajemen - Page 2/207 [------------------------------] 0.97%

In [4]:
scrape_prodi(7)

Prodi terdeteksi: Manajemen (ID: 7)
Ditemukan 207 halaman untuk di-scrape.
Scraping Manajemen - Page 207/207 [██████████████████████████████] 100.00%

✅ Selesai! Data prodi Manajemen berhasil dikumpulkan.
📄 Data disimpan dalam file: pta_manajemen_7.csv
📊 Total entri: 1031
⏱️ Waktu eksekusi: 0 jam 21 menit 7 detik


,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,080211100070,SATIYAH,PENGARUH FAKTOR-FAKTOR PELATIHAN DAN PENGEMBAN...,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...",ABSTRACT\r\n\r\nIn an effort to increase labor...,"Dra. Hj. S. Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST.SE,M.MT",Manajemen
1,090211200001,Faishal,ANALISIS PERSEPSI BRAND ASSOCIATION MENURUT PE...,Tujuan penelitian ini adalah untuk mengetahui ...,This study wanted to know the brand associatio...,Nurita Andriani,Yustina Chrismardani,Manajemen
2,080211100050,Wahyu Kurniawan,PENGARUH GAYA KEPEMIMPINAN DEMOKRATIK TERHADAP...,,,"Dr. Dra. Hj. Iriani Ismail, MM","Dra. Hj. S. Anugrahini Irawati, MM",Manajemen
3,100211200002,Muhammad Zakaria Utomo,Pengukuran Website Quality Pada Situs Sistem A...,Aplikasi nyata pemanfaatan teknologi informasi...,Academic portal system in University of Trunoj...,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Manajemen
4,080211100044,Hendri Wahyudi Prayitno,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Abstrak\r\nPenelitian ini menggunakan metode k...,Abstract\r\nThis research use quantitative met...,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Manajemen
...,...,...,...,...,...,...,...,...
1026,160211100071,Husnul Hotimah,Analisis Cost Volume Profit Untuk Menentukan T...,ABSTRAK\nPenelitian ini bertujuan untuk menget...,ABSTRACT\nThis study aims to determine the cal...,"Hj. Evaliati Amaniyah, S.E., M.S.M.",,Manajemen
1027,160211100291,Uswatun Hasanah,Pengaruh Pelatihan Dan Kompensasi Terhadap Pro...,"ABSTRAK\nUswatun Hasanah, 160211100291, Pengar...","ABSTRACT\nUswatun Hasanah, 160211100291, The E...","Dr. Raden Mas Mochammad Wispandono S.E ., MS",,Manajemen
1028,160211100064,ACH FATHONI,PERAN SERVICE PERFORMANCE DAN CLIMATE ORGANIZA...,ABSTRAK\nTujuan dari penelitian ini adalah unt...,ABSTRACK\n The purpose of this study...,"YUDHI PRASETYA MADA, S.E., M.M.",,Manajemen
1029,160211100030,INTAN YULLIA NINGSIH,BAURAN PROMOSI PADA DEALER YAMAHA TRETAN MOTOR...,ABSTRAK\nPenelitian ini bertujuan: (1) Untuk m...,ABSTRACK\nThis study aims: (1) To find out whe...,"DR. MOHAMMAD ARIEF, S.E., M.M.",,Manajemen


In [ ]:
import pandas as pd

# Load data from pta_manajemen.csv
df = pd.read_csv('pta_manajemen.csv')